In [ ]:
#INTEGRANTES: LUISA MARÍA HERNÁNDEZ QUINTERO, CAROLINA TABORDA VARGAS
import pandas as pd
import numpy as np
from scipy import stats

# Leer archivo Excel y usar la segunda fila como encabezado
df = pd.read_excel("FLIR_groups1and2.xlsx", header=1)

# Verificar que es un DataFrame
print(type(df))         

print(df.head())


In [ ]:
#Análisis exploratorio de datos

#Promedio de la variable Max1R13_
df = pd.read_excel("FLIR_groups1and2.xlsx", header=2)
columna=df[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4']].mean(axis=1)
print(f"El promedio de Max1R13_ es: \n {columna}")

#Tipos de datos: Variables numéricas y Variables categóricas
variables_numericas = ['aveOralM', 'Age', 'T_atm', 'Humidity', 'Max1R13_']
variables_categoricas = ['Gender', 'Ethnicity', 'Cosmetics']


In [ ]:
#Construcción 